In [194]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings ("ignore")

In [197]:
#Lectura y creación de dataframes
df_2020= pd.read_excel('2020.xlsx')
df_2021= pd.read_excel('2021.xlsx')
df_2022= pd.read_excel('2022.xlsx')

In [198]:
#Identificamos las columnas de la tabla mas recientes (2022), ya que va a ser nuestra base, el tipo de variables y cuantos nulos hay . 
df_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39479 entries, 0 to 39478
Data columns (total 54 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Nombre                                               39479 non-null  object 
 1   Código NIF                                           39479 non-null  object 
 2   Localidad                                            39478 non-null  object 
 3   Provincia                                            39479 non-null  object 
 4   País                                                 39479 non-null  object 
 5   Código primario CNAE 2009                            39479 non-null  int64  
 6   Literal código CNAE 2009 primario                    39479 non-null  object 
 7   Fecha constitución                                   39465 non-null  object 
 8   Importador / Exportador                              39479 non-nul

De aquí concluimos que tenemos que formatear la columna Fecha de constitución y que hay valores nulos.

## Procesamiento

### Tablas de 2020 y 2021

Descartamos la tabla del año 2021, ya que tiene una gran cantidad de duplicados y como vemos a continuación faltan bastantes empresas que estan en las tablas de 2020 y del 2022 (razonamiento más detallado en el informe).

In [199]:
# Comprobamos de cuantas empresas en común en las tres tablas y si eliminamos la tabla del 2021.
nif_comunes_total= set(df_2020['Código NIF'])&set(df_2022['Código NIF'])&set(df_2021['Código NIF'])
print(len(nif_comunes_total))
nif_comunes_20_22 =  set(df_2020['Código NIF'])&set(df_2022['Código NIF'])
print(len(nif_comunes_20_22))

27221
39464


### Tabla 2020

In [200]:
#Elegimos las columnas que necesitamos.
df_2020_ingexp = df_2020[['Código NIF','Ingresos de explotación\nmil EUR\n2020']]

In [201]:
#Tratamiento de Nans
conteo_fin = (df_2020_ingexp.applymap(lambda x: x == 'Financial Export limit reached')
              .sum()
              .sum())

print(f'Número de valores "Financial" en el DataFrame: {conteo_fin}')

conteo_nd = (df_2020_ingexp.applymap(lambda x: x == 'n.d.')
              .sum()
              .sum())

print(f'Número de valores "n.d" en el DataFrame: {conteo_nd}')

#Reemplazmos por NaNs
df_2020_ingexp.replace('n.d.',np.nan,inplace=True)
df_2020_ingexp.replace('Financial Export limit reached',np.nan,inplace=True)
df_2020_ingexp.isna().sum()

Número de valores "Financial" en el DataFrame: 2494


Número de valores "n.d" en el DataFrame: 7574


Código NIF                                    0
Ingresos de explotación\nmil EUR\n2020    10068
dtype: int64

In [202]:
#Duplicados. 
duplicados = df_2020_ingexp.duplicated()

# Mostrar los duplicados. No hay.
print(df_2020_ingexp[duplicados])

Empty DataFrame
Columns: [Código NIF, Ingresos de explotación
mil EUR
2020]
Index: []


### Tabla 2022

In [203]:
#Primero hemos decidido los ratios y en base a eso selecionamos las columnas que necesitamos para calcularlos + las columnas que consideramos relevantes para el clustering.
df_2022_col=df_2022[['Nombre', 'Código NIF', 'Localidad',
       'Provincia', 'País', 'Código primario CNAE 2009',
       'Literal código CNAE 2009 primario', 'Fecha constitución',
       'Importador / Exportador', 'Países importación', 'Países exportación','Resultado del Ejercicio\nmil EUR\n2022',
       'Total activo\nmil EUR\n2022', 'Ingresos de explotación\nmil EUR\n2022', 'Activo circulante\nmil EUR\n2022', 
       'Pasivo líquido\nmil EUR\n2022', 'Resultado Explotación\nmil EUR\n2022',  
       'Otros activos líquidos\nmil EUR\n2022','Capital suscrito\nmil EUR\n2022',
       'Otros fondos propios\nmil EUR\n2022','Número empleados\n2022', 'Pasivo fijo\nmil EUR\n2022',
       'Acreedores a L. P.\nmil EUR\n2022',
       'Otros pasivos fijos\nmil EUR\n2022',
       'Deudas financieras\nmil EUR\n2022',
       'Acreedores comerciales\nmil EUR\n2022',
       'Otros pasivos líquidos\nmil EUR\n2022', 'Inmovilizado\nmil EUR\n2022', 'Otros activos fijos\nmil EUR\n2022']]
df_2022_col.head()

,Nombre,Código NIF,Localidad,Provincia,País,Código primario CNAE 2009,Literal código CNAE 2009 primario,Fecha constitución,Importador / Exportador,Países importación,...,Otros fondos propios\nmil EUR\n2022,Número empleados\n2022,Pasivo fijo\nmil EUR\n2022,Acreedores a L. P.\nmil EUR\n2022,Otros pasivos fijos\nmil EUR\n2022,Deudas financieras\nmil EUR\n2022,Acreedores comerciales\nmil EUR\n2022,Otros pasivos líquidos\nmil EUR\n2022,Inmovilizado\nmil EUR\n2022,Otros activos fijos\nmil EUR\n2022
0,IBERDROLA CLIENTES SOCIEDAD ANONIMA.,A95758389,BILBAO,Vizcaya,ESPANA,3514,Comercio de energía eléctrica,2014-03-17 00:00:00,Exportador,NaN,...,1344809.0,887.0,73692.0,15522.0,58170.0,53247.0,210709.0,1414596.0,483676.0,315025.0
1,PAN AMERICAN ENERGY SL.,B88260203,BILBAO,Vizcaya,ESPANA,6420,Actividades de las sociedades holding,1997-09-29 00:00:00,Exportador,NaN,...,5777889.0,4939.0,4850463.0,1669228.0,3181235.0,369521.0,643621.0,714128.0,12748223.0,1732003.0
2,IBERDROLA ENERGIA ESPAÑA SOCIEDAD ANONIMA.,A95758371,BILBAO,Vizcaya,ESPANA,4611,Intermediarios del comercio de materias primas...,2014-03-17 00:00:00,Exportador,NaN,...,-772285.0,243.0,3751684.0,3736825.0,14859.0,117.0,190448.0,2028725.0,3301347.0,3255329.0
3,ALVEAN SUGAR SOCIEDAD LIMITADA.,B95773057,BILBAO,Vizcaya,ESPANA,4636,"Comercio al por mayor de azúcar, chocolate y c...",2014-07-17 00:00:00,Importador / Exportador,NaN,...,312183.0,38.0,4498.0,3878.0,620.0,284983.0,90500.0,112227.0,36129.0,31093.0
4,"IBERDROLA, SOCIEDAD ANONIMA",A48010615,BILBAO,Vizcaya,ESPANA,6420,Actividades de las sociedades holding,1901-07-19 00:00:00,Exportador,NaN,...,26909000.0,853.0,11434000.0,10099000.0,1335000.0,1170000.0,NaN,9058000.0,50042000.0,49653000.0


In [204]:
#Creamos un dataframe de 2022 con los niff en comun como la tabla de 2020
df_2022_col_comunes =  df_2022_col[df_2022_col['Código NIF'].isin(nif_comunes_20_22)]

In [205]:
#Tratamiento de Nans. No hace falta.
conteo_fin = (df_2022_col_comunes.applymap(lambda x: x == 'Financial Export limit reached')
              .sum()
              .sum())

print(f'Número de valores "Financial" en el DataFrame: {conteo_fin}')

conteo_nd = (df_2022_col_comunes.applymap(lambda x: x == 'n.d.')
              .sum()
              .sum())

print(f'Número de valores "n.d" en el DataFrame: {conteo_nd}')

Número de valores "Financial" en el DataFrame: 0
Número de valores "n.d" en el DataFrame: 0


In [231]:
#Duplicados. 
duplicados = df_2022_col_comunes.duplicated()

# Mostrar los duplicados. No hay.
print(df_2022_col_comunes[duplicados])

Empty DataFrame
Columns: [Nombre, Código NIF, Localidad, Provincia, País, Código primario CNAE 2009, Literal código CNAE 2009 primario, Fecha constitución, Importador / Exportador, Países importación, Países exportación, Resultado del Ejercicio
mil EUR
2022, Total activo
mil EUR
2022, Ingresos de explotación
mil EUR
2022, Activo circulante
mil EUR
2022, Pasivo líquido
mil EUR
2022, Resultado Explotación
mil EUR
2022, Otros activos líquidos
mil EUR
2022, Capital suscrito
mil EUR
2022, Otros fondos propios
mil EUR
2022, Número empleados
2022, Pasivo fijo
mil EUR
2022, Acreedores a L. P.
mil EUR
2022, Otros pasivos fijos
mil EUR
2022, Deudas financieras
mil EUR
2022, Acreedores comerciales
mil EUR
2022, Otros pasivos líquidos
mil EUR
2022, Inmovilizado
mil EUR
2022, Otros activos fijos
mil EUR
2022]
Index: []

[0 rows x 29 columns]


In [207]:
#Nos aseguramos de que estan todas las columnas que necesitamos
df_2022_col_comunes.columns

Index(['Nombre', 'Código NIF', 'Localidad', 'Provincia', 'País',
       'Código primario CNAE 2009', 'Literal código CNAE 2009 primario',
       'Fecha constitución', 'Importador / Exportador', 'Países importación',
       'Países exportación', 'Resultado del Ejercicio\nmil EUR\n2022',
       'Total activo\nmil EUR\n2022', 'Ingresos de explotación\nmil EUR\n2022',
       'Activo circulante\nmil EUR\n2022', 'Pasivo líquido\nmil EUR\n2022',
       'Resultado Explotación\nmil EUR\n2022',
       'Otros activos líquidos\nmil EUR\n2022',
       'Capital suscrito\nmil EUR\n2022',
       'Otros fondos propios\nmil EUR\n2022', 'Número empleados\n2022',
       'Pasivo fijo\nmil EUR\n2022', 'Acreedores a L. P.\nmil EUR\n2022',
       'Otros pasivos fijos\nmil EUR\n2022',
       'Deudas financieras\nmil EUR\n2022',
       'Acreedores comerciales\nmil EUR\n2022',
       'Otros pasivos líquidos\nmil EUR\n2022', 'Inmovilizado\nmil EUR\n2022',
       'Otros activos fijos\nmil EUR\n2022'],
      dtype=

In [208]:
#Analizamos cuantos valores nulos por columnas tenemos.
print(df_2022_col_comunes.isna().sum())

Nombre                                        0
Código NIF                                    0
Localidad                                     1
Provincia                                     0
País                                          0
Código primario CNAE 2009                     0
Literal código CNAE 2009 primario             0
Fecha constitución                           14
Importador / Exportador                       0
Países importación                        37019
Países exportación                        38496
Resultado del Ejercicio\nmil EUR\n2022    10253
Total activo\nmil EUR\n2022               10106
Ingresos de explotación\nmil EUR\n2022    11011
Activo circulante\nmil EUR\n2022          10121
Pasivo líquido\nmil EUR\n2022             10341
Resultado Explotación\nmil EUR\n2022      10217
Otros activos líquidos\nmil EUR\n2022     10509
Capital suscrito\nmil EUR\n2022           10107
Otros fondos propios\nmil EUR\n2022       10134
Número empleados\n2022                  

In [209]:
#Hemos descubierto que los registros que tienen NaN en el Total Activo tambien tienen NaN en el resto de columnas
#Procedemos a eliminar todos los registros que tengan valores nulos en la columnas Total Activo
df_2022_col_comunes=df_2022_col_comunes[df_2022_col_comunes['Total activo\nmil EUR\n2022'].notna()]
df_2022_col_comunes.isna().sum()

Nombre                                        0
Código NIF                                    0
Localidad                                     0
Provincia                                     0
País                                          0
Código primario CNAE 2009                     0
Literal código CNAE 2009 primario             0
Fecha constitución                            5
Importador / Exportador                       0
Países importación                        27301
Países exportación                        28557
Resultado del Ejercicio\nmil EUR\n2022      147
Total activo\nmil EUR\n2022                   0
Ingresos de explotación\nmil EUR\n2022      905
Activo circulante\nmil EUR\n2022             15
Pasivo líquido\nmil EUR\n2022               235
Resultado Explotación\nmil EUR\n2022        111
Otros activos líquidos\nmil EUR\n2022       403
Capital suscrito\nmil EUR\n2022               1
Otros fondos propios\nmil EUR\n2022          28
Número empleados\n2022                  

In [210]:
#Hacemos el merge de las tablas 2022 y 2020
df_2020_2022=df_2022_col_comunes.merge(df_2020_ingexp, on='Código NIF', how='left')

### Procesamiento por columnas

In [211]:
#COL: FECHA CONSTITUCIÓN
#Factorizar la columna
df_2020_2022['Fecha constitución'] = pd.to_datetime(df_2020_2022['Fecha constitución'])
df_2020_2022['Empresa reciente'] = df_2020_2022['Fecha constitución'].apply(lambda x: False if x.year < 2019 else True)

In [212]:
#COL: PROVINCIA
#Codificación de la columna en variables binarias
print(df_2020_2022['Provincia'].unique())
df_2020_2022 = pd.get_dummies(df_2020_2022,columns=["Provincia"])


['Vizcaya' 'Guipuzcoa' 'Alava']


In [213]:
#COL: CNAE (sectores)
# Función para mapear sectores: agrupados en 7 + "otros"
def mapeo_sectores(cnae):
    if cnae < 500:
        return "Otros" # Agricultura
    elif cnae < 1000:
        return "Otros" # Extraccion
    elif cnae < 3500:
        return "Industria"
    elif cnae < 3599:
        return "Otros" # Energia
    elif cnae < 4000:
        return "Otros" # Agua
    elif cnae < 4400:
        return "Construccion e Inmobilarias" # Construccion
    elif cnae < 4800:
        return "Comercio"
    elif cnae < 5500:
        return "Transporte"
    elif cnae < 5800:
        return "Hosteleria"
    elif cnae < 6400:
        return "Otros" # Comunicaciones
    elif cnae < 6800:
        return "Actividades Profesionales y Financieras" # Financieras
    elif cnae < 6900:
        return "Construccion e Inmobilarias" # Inmobiliarias
    elif cnae < 7700:
        return "Actividades Profesionales y Financieras" # Actividades Profesionales
    elif cnae < 8300:
        return "Otros" # Actividades Administrativas
    elif cnae < 8500:
        return "Otros" # Administracion Publica y Defensa
    elif cnae < 8600:
        return "Otros" # Educacion
    elif cnae < 9000:
        return "Otros" # Actividades Sanitarias
    elif cnae < 9400:
        return "Otros" # Actividades Artisticas
    else:
        return "Otros"
    
df_2020_2022["Sectores"] = df_2020_2022["Código primario CNAE 2009"].apply(mapeo_sectores)
df_2020_2022["Sectores"].value_counts()


Sectores
Construccion e Inmobilarias                6582
Comercio                                   6031
Otros                                      5115
Actividades Profesionales y Financieras    4544
Industria                                  4172
Hosteleria                                 1693
Transporte                                 1221
Name: count, dtype: int64

In [214]:
#Codificación de la columna en variables binarias
df_2020_2022 = pd.get_dummies(df_2020_2022,columns=["Sectores"])

In [215]:
#COL: Importador y Exportador
print(df_2020_2022["Importador / Exportador"].value_counts())

#Lista de Países que aparecen en las dos columnas
list(set(list(set((";".join(list(df_2020_2022["Países exportación"].value_counts().index))).split(";")))+ list(set((";".join(list(df_2020_2022["Países importación"].value_counts().index))).split(";")))))

#Lista de paises que NO utilizan EURO
no_euro = ["ESLOVAQUIA",
 'ARGENTINA',
 'FILIPINAS',
 'COSTA DE MARFIL',
 'AUSTRALIA',
 'ISRAEL',
 'URUGUAY',
 'CHINA',
 'SUECIA',
 'ORIENTE MEDIO',
 'SUDAMERICA',
 'CANADA',
 'MAURITANIA',
 'NORTE AMERICA',
 'CUBA',
 'DINAMARCA',
 'ARGELIA',
 'NIGERIA',
 'GHANA',
 'VENEZUELA',
 'SUI',
 'ECUADOR',
 'SINGAPUR',
 'NUEVA ZELANDA',
 'GROENLANDIA',
 'ARABIA SAUDI',
 'POLONIA',
 'SUIZA',
 'PAISES ARABES',
 'PAKISTAN',
 'HUNGRIA',
 'BAHREIN',
 'VIETNAM',
 'AFRICA DEL NORTE',
 'MOZAMBIQUE',
 'QATAR',
 'COREA DEL SUR',
 'TAILANDIA',
 'PERU',
 'MALI',
 'MEXICO',
 'PANAMA',
 'MALASIA',
 'UK',
 'RUMANIA',
 'OCEANIA',
 'EMIRATOS ARABES UNIDOS',
 'ASIA',
 'REPUBLICA CHECA',
 'AFRICA',
 'LIBIA',
 'BULGARIA',
 'AMERICA',
 'CHILE',
 'CAMERUN',
 'SERBIA',
 'IRAN',
 'TUNEZ',
 'COSTA RICA',
 'KAZAJSTAN',
 'BRASIL',
 'REPUBLICA DOMINICANA',
 'HONG KONG',
 'SENEGAL',
 'INDONESIA',
 'ANGOLA',
 'CENTRO AMERICA',
 'EGIPTO',
 'MARRUECOS',
 'EEUU',
 'RUSIA',
 'JAPON',
 'COLOMBIA',
 'TURQUIA',
 'UCRANIA',
 'SEYCHELLES',
 'INDIA',
 'SUDAFRICA']

Importador / Exportador
No realiza actividad exterior    24988
Importador / Exportador           1950
Importador                        1731
Exportador                         689
Name: count, dtype: int64


In [216]:
#Funcion para saber si la empresa opera con monedas que no sean EUROS
def pais(lista_paises):
    if type(lista_paises)==float:
        return np.nan
    lista_paises_sp = lista_paises.split(";")
    for pais in no_euro:
        if pais in lista_paises_sp:
            return True
    else:
        return False

exp = df_2020_2022["Países exportación"].apply(pais)
imp = df_2020_2022["Países importación"].apply(pais)

In [217]:
#Crear una lista de booleanos 
moneda = []
for i in range(len(exp)):
    if (exp[i] == True) or (imp[i]==True):
        moneda.append(True)
    else:
        moneda.append(False)

#Crear una nueva columna con los valores
df_2020_2022["Moneda no euro"] = moneda

In [218]:
df_2020_2022["Moneda no euro"].isna().sum() - df_2020_2022["Importador / Exportador"].value_counts()[0]

-24988

In [219]:
df_2020_2022[["Países importación","Países exportación","Moneda no euro"]].head(20)

,Países importación,Países exportación,Moneda no euro
0,NaN,UE,False
1,NaN,ARGENTINA,True
2,NaN,UE,False
3,NaN,NaN,False
4,NaN,UE,False
5,NaN,NaN,False
6,NaN,NaN,False
7,NaN,UE,False
8,NaN,NaN,False
9,UE,NaN,False


### RATIOS FINANCIERAS

In [220]:
#Endeudamiento
df_2020_2022['Otros fondos propios\nmil EUR\n2022'] = df_2020_2022['Otros fondos propios\nmil EUR\n2022'].fillna(0)
df_2020_2022['Pasivo Total']=df_2020_2022['Total activo\nmil EUR\n2022']-df_2020_2022['Capital suscrito\nmil EUR\n2022']-df_2020_2022['Otros fondos propios\nmil EUR\n2022']
columnas_sumar=['Pasivo fijo\nmil EUR\n2022', 'Pasivo líquido\nmil EUR\n2022']
df_2020_2022['Pasivo Total'] = df_2020_2022['Pasivo Total'].fillna(df_2020_2022[columnas_sumar].sum(axis=1))
df_2020_2022.loc[df_2020_2022['Pasivo Total'] < 0, 'Pasivo Total'] = 0
df_2020_2022['Endeudamiento']=df_2020_2022['Pasivo Total']/df_2020_2022['Total activo\nmil EUR\n2022']


In [221]:
#Solvencia
columnas_sumar2=['Inmovilizado\nmil EUR\n2022', 'Otros activos fijos\nmil EUR\n2022']
df_2020_2022['Activo circulante\nmil EUR\n2022']=df_2020_2022['Activo circulante\nmil EUR\n2022'].fillna(df_2020_2022['Total activo\nmil EUR\n2022'] - df_2020_2022[columnas_sumar2].sum(axis=1))
df_2020_2022['Pasivo fijo\nmil EUR\n2022']=df_2020_2022['Pasivo fijo\nmil EUR\n2022'].fillna(0)
df_2020_2022['Pasivo líquido\nmil EUR\n2022'] = df_2020_2022['Pasivo líquido\nmil EUR\n2022'].fillna(df_2020_2022['Pasivo Total'] - df_2020_2022['Pasivo fijo\nmil EUR\n2022'])
df_2020_2022['Solvencia']=df_2020_2022['Activo circulante\nmil EUR\n2022']/df_2020_2022['Pasivo líquido\nmil EUR\n2022']
df_2020_2022.loc[df_2020_2022['Solvencia'] < 0, 'Solvencia'] = 0

#Categorizar la variable
def solv(val):
    if val < 0.7:
        return 0
    elif val < 1:
        return 1
    elif val < 1.5:
        return 2
    else:
        return 3
    
df_2020_2022["Solvencia"] = df_2020_2022["Solvencia"].apply(lambda x: solv(x))

In [222]:
#Evolucion de la ventas
df_2020_2022['Evolucion_ventas_total']=df_2020_2022['Ingresos de explotación\nmil EUR\n2022']-df_2020_2022['Ingresos de explotación\nmil EUR\n2020']
df_2020_2022['Evolucion_ventas']=df_2020_2022['Evolucion_ventas_total']/df_2020_2022['Ingresos de explotación\nmil EUR\n2020']
df_2020_2022['Evolucion_ventas'] = df_2020_2022['Evolucion_ventas'].fillna(0)

In [223]:
#ROA
df_2020_2022['Resultado del Ejercicio\nmil EUR\n2022'] = df_2020_2022['Resultado del Ejercicio\nmil EUR\n2022'].fillna(0)
df_2020_2022['ROA']=df_2020_2022['Resultado del Ejercicio\nmil EUR\n2022']/df_2020_2022['Total activo\nmil EUR\n2022']

#### Selección de columnas necesarias para el modelo.

In [224]:
df_2020_2022.columns

Index(['Nombre', 'Código NIF', 'Localidad', 'País',
       'Código primario CNAE 2009', 'Literal código CNAE 2009 primario',
       'Fecha constitución', 'Importador / Exportador', 'Países importación',
       'Países exportación', 'Resultado del Ejercicio\nmil EUR\n2022',
       'Total activo\nmil EUR\n2022', 'Ingresos de explotación\nmil EUR\n2022',
       'Activo circulante\nmil EUR\n2022', 'Pasivo líquido\nmil EUR\n2022',
       'Resultado Explotación\nmil EUR\n2022',
       'Otros activos líquidos\nmil EUR\n2022',
       'Capital suscrito\nmil EUR\n2022',
       'Otros fondos propios\nmil EUR\n2022', 'Número empleados\n2022',
       'Pasivo fijo\nmil EUR\n2022', 'Acreedores a L. P.\nmil EUR\n2022',
       'Otros pasivos fijos\nmil EUR\n2022',
       'Deudas financieras\nmil EUR\n2022',
       'Acreedores comerciales\nmil EUR\n2022',
       'Otros pasivos líquidos\nmil EUR\n2022', 'Inmovilizado\nmil EUR\n2022',
       'Otros activos fijos\nmil EUR\n2022',
       'Ingresos de explot

In [225]:
df_procesado=df_2020_2022[["Código NIF", 'Provincia_Alava','Provincia_Guipuzcoa', 'Total activo\nmil EUR\n2022',
       'Empresa reciente', 'Sectores_Actividades Profesionales y Financieras',
       'Sectores_Comercio', 'Sectores_Construccion e Inmobilarias',
       'Sectores_Hosteleria', 'Sectores_Industria',
       'Sectores_Transporte', 'Moneda no euro',
       'Endeudamiento', 'Solvencia', 'Evolucion_ventas', 'ROA']]

In [226]:
df_procesado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29358 entries, 0 to 29357
Data columns (total 16 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Código NIF                                        29358 non-null  object 
 1   Provincia_Alava                                   29358 non-null  bool   
 2   Provincia_Guipuzcoa                               29358 non-null  bool   
 3   Total activo
mil EUR
2022                         29358 non-null  float64
 4   Empresa reciente                                  29358 non-null  bool   
 5   Sectores_Actividades Profesionales y Financieras  29358 non-null  bool   
 6   Sectores_Comercio                                 29358 non-null  bool   
 7   Sectores_Construccion e Inmobilarias              29358 non-null  bool   
 8   Sectores_Hosteleria                               29358 non-null  bool   
 9   Sectores_Industri

In [227]:
df_procesado.to_excel('df_procesado.xlsx', index=False)